# PCB-SuperNet Research Notebook

This notebook provides an end-to-end, research-friendly workflow to train and compare PCB defect models (VGG16, ResNet50, YOLOv10) and the proposed **PCB-MultiHead Fusion** super-model.


## 1. Environment Setup
Install required dependencies and import core packages.


In [ ]:
# !pip install kagglehub albumentations opencv-python torch torchvision ultralytics

import torch
from pathlib import Path

from pcb_supernet.config import DataConfig, ExperimentConfig, ModelConfig, TrainingConfig
from pcb_supernet.main import parse_class_names, run_experiment
from pcb_supernet.models import PCBMultiHeadFusion
from pcb_supernet.visualize import test_and_visualize
from pcb_supernet.baselines import build_vgg16, build_resnet50, build_yolov10


## 2. Download Datasets
Use KaggleHub to download HRIPCB_UPDATE, DeepPCB, and deeppcb datasets.


In [ ]:
import kagglehub

hripcb_path = Path(kagglehub.dataset_download("ibrahimgergesmoussa/hripcb-update"))
deeppcb_path = Path(kagglehub.dataset_download("dangdinh123/deeppcb"))
deeppcb_alt_path = Path(kagglehub.dataset_download("kcnttngotruongan/deeppcb"))

dataset_roots = [hripcb_path, deeppcb_path, deeppcb_alt_path]
print(dataset_roots)


## 3. Parse data.yaml for Class Consistency
Ensure class definitions are aligned across all datasets.


In [ ]:
class_names = parse_class_names(dataset_roots)
print(class_names)


## 4. Configure Experiment
Define data, model, and training configs.


In [ ]:
data_cfg = DataConfig(root_dirs=dataset_roots, class_names=class_names)
model_cfg = ModelConfig(num_classes=len(class_names))
train_cfg = TrainingConfig(epochs=20, lr=1e-4)
exp_cfg = ExperimentConfig(data=data_cfg, model=model_cfg, training=train_cfg)


## 5. Train PCB-MultiHead Fusion (Super-Model)
Train the late-fusion model and run validation metrics.


In [ ]:
run_experiment(exp_cfg)


## 6. Train Baselines
Example instantiation for VGG16/ResNet50 transfer learning baselines.
YOLOv10 requires `ultralytics` and its training pipeline.


In [ ]:
vgg16_model = build_vgg16(num_classes=len(class_names))
resnet50_model = build_resnet50(num_classes=len(class_names))
# yolo_model = build_yolov10()
# yolo_results = yolo_model.train(data='path/to/data.yaml', epochs=50)


## 7. Visualization (Grad-CAM)
Visualize predictions for 5 random test images.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fusion_model = PCBMultiHeadFusion(num_classes=len(class_names)).to(device)
# Load checkpoint if available:
# fusion_model.load_state_dict(torch.load('outputs/pcb_supernet.pt')['model_state'])
test_and_visualize(
    model=fusion_model,
    test_dir=dataset_roots[0] / 'test' / 'images',
    class_names=class_names,
    device=device,
)


## 8. Reporting Tables
Fill in Table 1 and Table 2 in `REPORT.md` after running experiments.
